In [ ]:
import extractor
from graphviz import Digraph

blocks={};
blockIndex=0;
edges=[];
prevBlock=-1;
nonBlankBlocks={};
blankBlock=[];
finalEdges=[];
blockLineNum={};
linesNum=[];
leaf=[]
whileStartEnd={}

#create basic blocks for body of if,while,module...
def getBody(lst,prevBlock):
    global blocks
    global blockIndex;
    global edges
    global leaf
    global linesNum
    global whileStartEnd

    block="";
    flag=1
    for item in lst:
        #Add Expressions in Basic Blocks
        if item['_type']=='Expr':
            edges.append(str(prevBlock)+":"+str(blockIndex));
            blocks[blockIndex]=block;
            blockLineNum[blockIndex]=linesNum;
            linesNum=[];
            prevBlock=blockIndex;
            blockIndex+=1;
            block=""
            block=extractor.getExpression(item)
            linesNum.append(getLineNum(item));
            blocks[blockIndex]=block;
            blockLineNum[blockIndex]=linesNum;
            edges.append(str(prevBlock)+":"+str(blockIndex));
            linesNum=[];
            block=""
            prevBlock=blockIndex;
            blockIndex+=1

        #Add Assign Statments in Basic Blocks
        elif item['_type']=='Assign':
            block+=extractor.getAssignCondition(item)+"\n";
            linesNum.append(getLineNum(item));

        #Add AugAssign Statments in Basic Blocks
        elif item['_type']=='AugAssign':
            block+=extractor.getAugAssignCondition(item)+"\n";
            linesNum.append(getLineNum(item));

        elif item['_type']=='Delete':
            block+=extractor.extractNodes(item)+"\n";
            linesNum.append(getLineNum(item));

        #Creates a Basic Block and call runForDict for if Statement
        elif item['_type']=='If':
            edges.append(str(prevBlock)+":"+str(blockIndex));
            blocks[blockIndex]=block;
            blockLineNum[blockIndex]=linesNum;
            linesNum=[];

            tempBlock=blockIndex;
            blockIndex+=1;
            prevBlock=blockIndex;
            
            runForDict(item,tempBlock);
            flag=0;
            block="";
            #print(leaf)
            #print(blockIndex)
            i= prevBlock
            for i in range(blockIndex):
                if str(i) in leaf:
                    #print("leaf found",i)
                    edges.append(str(i)+":"+str(blockIndex));

        #Creates a Basic Block and call runForDict for while Statement
        elif item['_type']=='While':
            edges.append(str(prevBlock)+":"+str(blockIndex));
            blocks[blockIndex]=block;
            blockLineNum[blockIndex]=linesNum;
            linesNum=[];

            tempBlock=blockIndex;
            blockIndex+=1;
            prevBlock=blockIndex;
            flag=1;

            runForDict(item,tempBlock);
            edges.append(str(blockIndex-1)+":"+str(prevBlock));
            whileStartEnd.update({prevBlock : blockIndex-1})
            block=""

    #Basic Block Created
    if flag:
        edges.append(str(prevBlock)+":"+str(blockIndex));
    blocks[blockIndex]=block;
    blockLineNum[blockIndex]=linesNum;
    linesNum=[];
    blockIndex+=1

#traverse for if, while, and Module blocks
def runForDict(node,prevBlock):
    global blocks
    global blockIndex;
    global edges
    global leaf
    global linesNum
    
    #Call for creation of Basic Blocks for Exp and Assign Statements
    if node['_type']=='Module':
        getBody(node['body'],prevBlock);
    
    elif node['_type']=='If':
        #Call for creation of Basic Blocks for branch condition
        edges.append(str(prevBlock)+":"+str(blockIndex));
        blocks[blockIndex]='branch['+extractor.getBranchStat(node)+']';
        blockLineNum[blockIndex]=[getLineNum(node)];
        prevBlock=blockIndex;
        blockIndex+=1;
        leaf.append(str(blockIndex))

        #Call for creation of Basic Blocks for body of if
        
        getBody(node['body'],prevBlock);
        if len(node['orelse'])>0:
            leaf.append(str(blockIndex))
            getBody(node['orelse'],prevBlock);
        else:
            leaf.append(str(prevBlock));
        # if (blockIndex-prevBlock)==2:
        #     edges.append(str(prevBlock)+":"+str(blockIndex));
        # elif (blockIndex-prevBlock)==3:
        #     edges.append(str(prevBlock+2)+":"+str(blockIndex));   
            

    elif node['_type']=='While':
        #Call for creation of Basic Blocks for while loop condition
        edges.append(str(prevBlock)+":"+str(blockIndex));
        blocks[blockIndex]='while['+extractor.getWhileConditionalStat(node)+']';
        blockLineNum[blockIndex]=[getLineNum(node)];
        prevBlock=blockIndex;
        blockIndex+=1;

        #Call for creation of Basic Blocks for body of while
        getBody(node['body'],prevBlock);

#Creates Basic Blocks (Nodes) and Edge List of CFG
def generateCFG(myDict):
    global prevBlock
    global edges
    global finalEdges
    global nonBlankBlocks

    runForDict(myDict,prevBlock);

    #Removing multiple Edegs between same nodes
    edges = list(dict.fromkeys(edges))

    #Remove Empty basic blocks
    for i,v in blocks.items():
        if len(v)!=0:
            nonBlankBlocks.update({i:v});
        else:
            blankBlock.append(str(i));

    """
    #basic blocks
    for i,v in nonBlankBlocks.items():
        print(i,v);
    """

    #Remove Empty block edges
    for i in edges:
        lst=i.split(':');
        if lst[0] in blankBlock:
            pass;
    
        elif lst[1] in blankBlock:
            for j in edges:
                lis=j.split(':');
                if lst[1]==lis[0]:
                    if lis[1] not in blankBlock:
                        finalEdges.append(str(lst[0])+":"+str(lis[1]));

                    else:
                        lst[1]=lis[1];
                        
        else:
            finalEdges.append(str(lst[0])+":"+str(lst[1]));

#CFG Rendering
def drawCFG():
    global nonBlankBlocks
    global finalEdges
    
    #Creating CFG digraph for Rendering
    cfg=Digraph(name='cfg',comment="control flow graph",)
    for i,v in nonBlankBlocks.items():
        cfg.node(str(i),label=str(v))

    for i in finalEdges:
        lst=i.split(':');
        if lst[0]!='-1':
            cfg.edge(str(lst[0]),str(lst[1]));

    cfg.render('cfg', view=True)
    print("CFG Created.")

#returns a Dict of Edge List
def getEdgeList():
    global finalEdges
    edgeList={};
    for item in nonBlankBlocks:
        edgeList[item]=[];

    for item in finalEdges:
        edgePair=item.split(':');
        if edgePair[0]!='-1':
            edgeList[int(edgePair[0])].append(int(edgePair[1]));

    return edgeList;

#returns a Dict Of Basic Blocks with their index as keys
def getBasicBlocks():
    global nonBlankBlocks

    return nonBlankBlocks;

def getLineNum(node):
    return node['lineno'];
def whilestartend():
    return whileStartEnd

def getLineDict():
    newLineDict={};
    for k,v in blockLineNum.items():
        if len(v)>0:
            newLineDict[k]=v;
    return newLineDict;
def clear():
    global blocks
    global blockIndex
    global edges
    global prevBlock
    global nonBlankBlocks
    global blankBlock
    global finalEdges
    global blockLineNum
    global linesNum
    global leaf
    global whileStartEnd
    blocks={};
    blockIndex=0;
    edges=[];
    prevBlock=-1;
    nonBlankBlocks={};
    blankBlock=[];
    finalEdges=[];
    blockLineNum={};
    linesNum=[];
    leaf=[]
    whileStartEnd={}